In [29]:
%pip install matplotlib opencv-python face_recognition librosa pillow scikit-image pillow-heif xgboost

#Data manipulation libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#ML libraries
import sklearn
import xgboost as xgb

# Image & Audio processing libraries
import cv2
from PIL import Image, ImageOps


Defaulting to user installation because normal site-packages is not writeable
  Using cached face_recognition-1.3.0-py2.py3-none-any.whl.metadata (21 kB)
  Using cached scikit_image-0.25.2-cp313-cp313-win_amd64.whl.metadata (14 kB)
  Using cached pillow_heif-1.1.1-cp313-cp313-win_amd64.whl.metadata (9.8 kB)
  Using cached face_recognition_models-0.3.0-py2.py3-none-any.whl
  Using cached dlib-20.0.0.tar.gz (3.3 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached networkx-3.5-py3-none-any.whl.metadata (6.3 kB)
  Using cached imageio-2.37.2-py3-none-any.whl.metadata (9.7 kB)
  Using cached tifffile-2025.10.16-py3-none-any.whl.metadata (31 kB)
Using cached face_recognition-1.3.0-py2.py3-none-any.wh

  error: subprocess-exited-with-error
  
  × Building wheel for dlib (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [41 lines of output]
      running bdist_wheel
      running build
      running build_ext
      
      
                         CMake is not installed on your system!
      
          Or it is possible some broken copy of cmake is installed on your system.
          It is unfortunately very common for python package managers to include
          broken copies of cmake.  So if the error above this refers to some file
          path to a cmake file inside a python or anaconda or miniconda path then you
          should delete that broken copy of cmake from your computer.
      
          Instead, please get an official copy of cmake from one of these known good
          sources of an official cmake:
              - cmake.org (this is how windows users should get cmake)
              - apt install cmake (for Ubuntu or Debian based systems)
             

## Loading Data

In [33]:

profiles_df = pd.read_csv('C:\\Users\\LENOVO\\Cloned repos\\ml-pipeline-formative\\ML-Pipeline-formative-2\\data\\datasets_to_merge\\customer_social_profiles - customer_social_profiles.csv')
transactions_df = pd.read_csv('C:\\Users\\LENOVO\\Cloned repos\\ml-pipeline-formative\\ML-Pipeline-formative-2\\data\\datasets_to_merge\\customer_transactions - customer_transactions.csv')
display(transactions_df.head(5))
display(profiles_df.head(5))

,customer_id,transaction_id,purchase_amount,purchase_date,product_category,customer_rating
0,151,1001,408,1/1/2024,Sports,2.3
1,192,1002,332,1/2/2024,Electronics,4.2
2,114,1003,442,1/3/2024,Electronics,2.1
3,171,1004,256,1/4/2024,Clothing,2.8
4,160,1005,64,1/5/2024,Clothing,1.3


,customer_id,social_media_platform,engagement_score,purchase_interest_score,review_sentiment
0,A178,LinkedIn,74,4.9,Positive
1,A190,Twitter,82,4.8,Neutral
2,A150,Facebook,96,1.6,Positive
3,A162,Twitter,89,2.6,Positive
4,A197,Twitter,92,2.3,Neutral


## Dataset Merge

In [34]:
profiles_df['customer_id'] = profiles_df['customer_id'].str.replace('A', '').astype(int)
merged_df = pd.merge(transactions_df, profiles_df, on='customer_id', how='inner')
merged_df.head(5)

,customer_id,transaction_id,purchase_amount,purchase_date,product_category,customer_rating,social_media_platform,engagement_score,purchase_interest_score,review_sentiment
0,151,1001,408,1/1/2024,Sports,2.3,TikTok,61,1.3,Neutral
1,151,1001,408,1/1/2024,Sports,2.3,Twitter,72,1.6,Neutral
2,151,1001,408,1/1/2024,Sports,2.3,Twitter,82,3.6,Negative
3,192,1002,332,1/2/2024,Electronics,4.2,Instagram,60,4.3,Positive
4,114,1003,442,1/3/2024,Electronics,2.1,Facebook,87,4.8,Negative


In [35]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 219 entries, 0 to 218
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   customer_id              219 non-null    int64  
 1   transaction_id           219 non-null    int64  
 2   purchase_amount          219 non-null    int64  
 3   purchase_date            219 non-null    object 
 4   product_category         219 non-null    object 
 5   customer_rating          200 non-null    float64
 6   social_media_platform    219 non-null    object 
 7   engagement_score         219 non-null    int64  
 8   purchase_interest_score  219 non-null    float64
 9   review_sentiment         219 non-null    object 
dtypes: float64(2), int64(4), object(4)
memory usage: 17.2+ KB


## Image Pipeline

In [ ]:
import pillow_heif

# ensure pillow_heif is registered so PIL can open HEIC files
pillow_heif.register_heif_opener()

def load_image(path=r'C:\Users\LENOVO\Cloned repos\ml-pipeline-formative\ML-Pipeline-formative-2\data\images\antony_neutral.heic', size=None):
    """
    Load an image (HEIC supported via pillow_heif).
    - path: path to the image (string)
    - size: optional (width, height) tuple to resize the image
    Returns a PIL.Image.Image instance.
    """
    image = Image.open(path)
    if size is not None:
        image = image.resize(size)
    return image

def augmen